In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Correlation Analysis
# a) Collect historical data for NASDAQ and NSE indices
nasdaq_data = yf.download('^IXIC', start='2010-01-01')
nse_data = yf.download('^NSEI', start='2010-01-01')

# b) Calculate the correlation coefficient between the two indices
correlation_coefficient = np.corrcoef(nasdaq_data['Close'], nse_data['Close'])[0, 1]

# c) Analyze the strength and direction of the relationship
if correlation_coefficient > 0:
    strength = 'strong positive'
elif correlation_coefficient < 0:
    strength = 'strong negative'
else:
    strength = 'no'

print("Correlation Analysis:")
print("Correlation Coefficient:", correlation_coefficient)
print("Strength of Relationship:", strength)


# Lead-Lag Relationship
# a) Identify potential lead-lag relationships between the indices
# To identify potential lead-lag relationships, you can visualize the data and observe patterns
nasdaq_data = pd.read_csv('nasdaq_data.csv')  # Replace with the actual filename or path
nse_data = pd.read_csv('nse_data.csv')  # Replace with the actual filename or path

# b) Analyze data to determine consistent leading or lagging behavior
correlation = np.correlate(nasdaq_data['Close'], nse_data['Close'], mode='full')
lag = np.argmax(correlation) - (len(nasdaq_data) - 1)
# Use statistical techniques like cross-correlation analysis or lagged correlation analysis to determine consistent lead-lag behavior
plt.plot(nasdaq_data['Date'], nasdaq_data['Close'], label='NASDAQ')
plt.plot(nse_data['Date'], nse_data['Close'], label='NSE')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Lead-Lag Relationship Analysis')
plt.show()

lagged_correlation = np.correlate(nasdaq_data['Close'].shift(lag), nse_data['Close'], mode='full')


# c) Use the lead-lag relationship to determine the index to be used for parameter optimization
primary_index = 'NASDAQ' if lag > 0 else 'NSE'

# Choose the index that consistently leads or lags as the reference index for parameter optimization

# d) Provide an explanation for choosing the index for parameter optimization
explanation = f"The {primary_index} index is chosen for parameter optimization as it consistently {('lags' if lag > 0 else 'leads')} the {('NSE' if lag > 0 else 'NASDAQ')} index."

print(explanation)
# Document the rationale behind choosing the leading or lagging index for parameter optimization


# Indicator Coding
def calculate_keltner_channel(data, window_size, multiplier):
    typical_price = (data['High'] + data['Low'] + data['Close']) / 3
    average_true_range = data['High'] - data['Low']
    upper_band = typical_price + (multiplier * average_true_range)
    lower_band = typical_price - (multiplier * average_true_range)
    return upper_band, lower_band

def calculate_bollinger_bands(data, window_size, std_multiplier):
    rolling_mean = data['Close'].rolling(window=window_size).mean()
    rolling_std = data['Close'].rolling(window=window_size).std()
    upper_band = rolling_mean + (std_multiplier * rolling_std)
    lower_band = rolling_mean - (std_multiplier * rolling_std)
    return upper_band, lower_band

def calculate_macd(data, short_period, long_period, signal_period):
    exp1 = data['Close'].ewm(span=short_period, adjust=False).mean()
    exp2 = data['Close'].ewm(span=long_period, adjust=False).mean()
    macd_line = exp1 - exp2
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    histogram = macd_line - signal_line
    return macd_line, signal_line, histogram

# Parameter Optimization
def optimize_parameters(data, indicator_func, parameter_range):
    best_metric = -np.inf
    best_parameter = None
    
    for parameter in parameter_range:
        upper_band, lower_band = indicator_func(data, parameter)
        # Perform strategy backtesting and calculate evaluation metrics
        # Daily Returns, Cumulative Returns, Max Drawdowns, Sharpe Ratio, Sortino Ratio
        
        # Update best parameter if current value yields better metric
        if metric > best_metric:
            best_metric = metric
            best_parameter = parameter
    
    return best_parameter

# Optimize parameters for Keltner Channel
keltner_channel_parameter_range = range(2, 6)
optimized_keltner_channel_parameter = optimize_parameters(nasdaq_data, calculate_keltner_channel, keltner_channel_parameter_range)

# Optimize parameters for Bollinger Bands
bollinger_bands_parameter_range = np.arange(1.5, 3.1, 0.5)
optimized_bollinger_bands_parameter = optimize_parameters(nasdaq_data, calculate_bollinger_bands, bollinger_bands_parameter_range)

# Optimize parameters for MACD
macd_short_period_range = range(6, 16)
macd_long_period_range = range(17, 31)
macd_signal_period_range = range(4, 10)
optimized_macd_parameters = optimize_parameters(nasdaq_data, calculate_macd, macd_short_period_range, macd_long_period_range, macd_signal_period_range)

# Document the optimized parameters for future reference
print("Parameter Optimization:")
print("Optimized Keltner Channel Parameter:", optimized_keltner_channel_parameter)
print("Optimized Bollinger Bands Parameter:", optimized_bollinger_bands_parameter)
print("Optimized MACD Parameters:", optimized_macd_parameters)


# Signal Generation
def generate_signals(data, indicator_func, parameter):
    signals = pd.DataFrame(index=data.index)
    signals['Signal'] = np.nan
    signals['Date'] = data.index
    signals['Return'] = 0.0
    
    # Generate buy and sell signals based on the indicator values and a trading strategy
    for i in range(len(data)):
        if i == 0:
            signals['Signal'].iloc[i] = 0  # No signal on the first day
        elif data['Indicator'].iloc[i-1] < data['Indicator'].iloc[i]:
            signals['Signal'].iloc[i] = 1  # Buy signal
        elif data['Indicator'].iloc[i-1] > data['Indicator'].iloc[i]:
            signals['Signal'].iloc[i] = -1  # Sell signal
        
        if signals['Signal'].iloc[i] == 1:
            signals['Return'].iloc[i] = data['Close'].iloc[i] - data['Close'].iloc[i-1]  # Calculate return for buy signal
        elif signals['Signal'].iloc[i] == -1:
            signals['Return'].iloc[i] = data['Close'].iloc[i-1] - data['Close'].iloc[i]  # Calculate return for sell signal
    
    signals['Cumulative Return'] = signals['Return'].cumsum()
    signals['Max Drawdown'] = signals['Cumulative Return'].cummax() - signals['Cumulative Return']
    
    return signals

# Apply optimized parameters to the other index and generate buy/sell signals
nse_signals = generate_signals(nse_data, calculate_keltner_channel, optimized_keltner_channel_parameter)

# Record the signals, their respective dates, and returns along with other metrics
nse_data_with_signals = nse_data.copy()
nse_data_with_signals['Signal'] = nse_signals['Signal']
nse_data_with_signals['Return'] = nse_signals['Return']
nse_data_with_signals['Cumulative Return'] = nse_signals['Cumulative Return']
nse_data_with_signals['Max Drawdown'] = nse_signals['Max Drawdown']

# Print the recorded signals, their respective dates, returns, and other metrics for the other index
print("NSE Signals:")
print(nse_data_with_signals)

# Continue with further analysis, visualization, and backtesting as necessary
# Apply optimized parameters to the NASDAQ index and generate buy/sell signals
nasdaq_signals = generate_signals(nasdaq_data, calculate_keltner_channel, optimized_keltner_channel_parameter)

# Record the signals, their respective dates, and returns along with other metrics
nasdaq_data_with_signals = nasdaq_data.copy()
nasdaq_data_with_signals['Signal'] = nasdaq_signals['Signal']
nasdaq_data_with_signals['Return'] = nasdaq_signals['Return']
nasdaq_data_with_signals['Cumulative Return'] = nasdaq_signals['Cumulative Return']
nasdaq_data_with_signals['Max Drawdown'] = nasdaq_signals['Max Drawdown']

# Print the recorded signals, their respective dates, returns, and other metrics for the NASDAQ index
print("NASDAQ Signals:")
print(nasdaq_data_with_signals)